In [9]:
import os
import json
from dotenv import load_dotenv
from pprint import pprint as pp
load_dotenv()

with open('../data/extracted_books/1 - Dune - Frank Herbert.json', 'r') as f:
    data = json.load(f)


In [21]:
from pprint import pprint as pp

def extract_content(data, result=None):
    if result is None:
        result = []
    for item in data:
        result.append(item['content'])
        if 'children' in item:
            extract_content(item['children'], result)
    return result

text_parts = extract_content(data['data'])
sub_parts = [sub_part for text_part in text_parts[7:65] for sub_part in text_part.split('\n\n\n')]


In [25]:
from openai import OpenAI
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

summary_prompt = "You will be given a part of a book chapter between the <TEXT_PART> and </TEXT_PART> tags, you will summarize the actions that occur in that text. You will only output the summary, nothing else, keep it brief and factual, don't add any external knowledge, here is the text part: \n <TEXT_PART> {text_part} </TEXT_PART>"

entity_extraction_prompt = "You will be given a part of a book chapter between the <TEXT_PART> and </TEXT_PART> tags. You will extract every named entity in that text and classify them as PERSON, LOCATION, ORGANIZATION or CONCEPT, don't add any external knowledge. You will output your answer in the following format : \n <entity> : <category> \n for example : Jean : PERSON \n The great wall : LOCATION \n Mireille : PERSON \n Here is the text part: \n <TEXT_PART> {text_part} </TEXT_PART>"

entity_summarization_prompt = """
You will be given a part of a book chapter between the <TEXT_PART> and </TEXT_PART> tags. You will extract every named entity in that text and classify them as PERSON, LOCATION, ORGANIZATION or CONCEPT. Here is a description of each entity type :
PERSON : any character, can be human or not, real or imaginary
LOCATION : any physical location, can be any type of place, that is specific to the story
ORGANIZATION : any group a people that operates as a cohesive unit under a specific name
CONCEPT : notion or idea specific to the story

You will also add a short sentence about new information that we can gain about the entity by reading the text part, adopt a confident tone. Write this as if it were an entry in an encyclopedia, in english.
Don't add any external knowledge. I there is no entity in the text, output the following tag : <NONE>
You will output your answer in the following format, one entity per line :
entity | category | summary of what we learn

<TEXT_PART>
{text_part}
</TEXT_PART>
"""

for sub_part in sub_parts[:10]:
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "user", "content": entity_summarization_prompt.format(text_part=sub_part)}
      ]
    )
    print(sub_part)
    print('---')
    print(completion.choices[0].message.content)
    print('--------------------')

DUNE
---
DUNE | CONCEPT | The term "Dune" refers to the central theme in the story, typically signifying a significant desert landscape that plays a crucial role in the narrative.
--------------------
'est à l'heure du commencement qu'il faut tout particulièrement veiller à ce que les équilibres soient précis. Et cela, chaque sœur du Bene Gesserit le sait bien. Ainsi, pour entreprendre cette étude de la vie de Muad'Dib, il convient de le placer tout d'abord en son temps, en la cinquante-septième année de l'Empereur Padishah, Shaddam IV. Il convient aussi de bien le situer, sur la planète Arrakis. Et l'on ne devra pas se laisser abuser par le fait qu'il naquit sur Caladan et y vécut les quinze premières années de sa vie : Arrakis, la planète connue sous le nom de Dune restera sienne à jamais.

Extrait du
Manuel de Muad'Dib
par la Princesse Irulan
.
---
Muad'Dib | PERSON | Muad'Dib is a central figure in the narrative, whose life is pivotal to the unfolding events of the story.

Bene Ges